# Enhancing RAG with Neo4j Knowledge Graph


Import modules and instantiate connections and models

In [2]:
import fundus

import config
from graph import NewsGraphClient, URI
from ner import EntityFinder

/home/djordje/.virtualenvs/bbuzz24/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Crawl news sites
publishers = (fundus.PublisherCollection.de, fundus.PublisherCollection.uk)
crawler = fundus.Crawler(*publishers)
articles = list(crawler.crawl(max_articles=2))
for article in articles:
    print(article)

Fundus-Article:
- Title: "Keine Angst vor dem ersten Schultag: So finden Eltern einen sicheren [...]"
- Text:  "Zu Fuß, mit dem Fahrrad, oder mit Bus und Bahn: Wir zeigen, wie Eltern einen
          sicheren Schulweg für ihre Kinder finden.  Für hunderttausende [...]"
- URL:    https://www1.wdr.de/nachrichten/sicherer-schulweg-tipps-vor-schulstart100.html
- From:   Westdeutscher Rundfunk (2024-08-03 06:00)
Fundus-Article:
- Title: "Schulstart: Erstklässler in Mitteldeutschland feiern Einschulung"
- Text:  "Für viele Kinder ist es einer der wichtigsten Tage des Lebens: die
          Schuleinführung. Auch für die Eltern ist der Tag bedeutend, denn [...]"
- URL:    https://www.mdr.de/nachrichten/deutschland/gesellschaft/schulanfang-mitteldeutschland-einschulung-100.html
- From:   Mitteldeutscher Rundfunk (MDR) (2024-08-03 12:54)


## Perform some queries
e.g. list a few sources

In [5]:
import os
print(os.getenv('DB_URL'))
# import dotenv
# dotenv.load_dotenv()
db = NewsGraphClient()
# records = db.query("MATCH (s:Source) RETURN s.name,  as source LIMIT $limit", limit=10)
# for record in records:
#     print(record)

neo4j+s://dfa2b95c.databases.neo4j.io


## Perform Named Entity Recognition
with GLiNER

In [6]:
entity_finder = EntityFinder(config.RELEVANT_LABELS)

for article in articles:   
    entities_in_headline = entity_finder.find(article.title)
    print(article.title, 'mentions the following entities:', *(f"{e.label}: {e.name} |" for e in entities_in_headline))
    entities_in_text = entity_finder.find(article.plaintext)
    print('The text also mentions the following entities:', *(f"{e.label}: {e.name} |" for e in entities_in_text))

/home/djordje/.virtualenvs/bbuzz24/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/djordje/.virtualenvs/bbuzz24/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Keine Angst vor dem ersten Schultag: So finden Eltern einen sicheren Schulweg für ihre Kinder mentions the following entities: person: Eltern |


/home/djordje/.virtualenvs/bbuzz24/lib/python3.10/site-packages/gliner/modules/data.py:93: UserWarning: Sentence of length 985 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


The text also mentions the following entities: person: Eltern | location: Westen | person: Eltern | person: Eltern | organization: Landesverkehrswacht NRW | person: Präsident Klaus Voussem | person: Eltern | organization: ADAC | organization: ADAC | person: Eltern |
Schulstart: Erstklässler in Mitteldeutschland feiern Einschulung mentions the following entities: person: Erstklässler | location: Mitteldeutschland |
The text also mentions the following entities: location: Mitteldeutschland | location: Sachsen | location: Sachsen-Anhalt | location: Thüringen | location: Mitteldeutschland | location: Sachsen | location: Sachsen-Anhalt | location: Thüringen | location: Sachsen-Anhalt | organization: Bildungsministerium | organization: CDU | organization: SPD | organization: FDP | location: Sachsen | person: Kultusminister Christian Piwarz | location: Thüringen | person: Helmut Holter | organization: Linke |
